In [ ]:
# Test Cell
# When the notebook is executed by the widget, this cell is ignored. 
import pandas as pd
import sys
import base64
import io
import os

additional_parameters = "{\"question\":\"dashboard Sample - Healthcare widget TOP 10 DIAGNOSIS\""+\
",\"to_validate_entities_from_question\": false,\"widgetId\":\"63ac28e77a19050033efd92d\",\"model_name\":\"Sample Healthcare\"" + \
",\"cookie\":\"XXX\"}"


df_input = pd.DataFrame()
csv_input =  "QmVmb3JlXG5DT08="
print('size of csv_input is:' + str(sys.getsizeof(csv_input)) + ' bytes')
bytes = base64.b64decode(csv_input).decode('unicode_escape')
df_input = pd.read_csv(io.StringIO(bytes), na_values='!#NULL#!')

# for local develop
# # Insert IP of Sisense instance
# os.environ['API_GATEWAY_EXTERNAL_SERVICE_HOST'] = "X.X.X.X"
# # Insert Port of Sisense instance
# os.environ['API_GATEWAY_EXTERNAL_SERVICE_PORT'] = "30845"
# # Insert Cookie received by Sisense REST API - '/authentication/tokens/api'
# os.environ['Cookie'] = "XXX"

# Getting Started

Welcome to the AI-model (GPT) Widget Summary Notebook.  

This notebook shows how to summarize widget data with GPT model. Using this notebook it is possible to:
1. retrieve widget data in Sisense and generate summary with GPT model.


Widget summarization from Sisense can be done in 2 ways:
1. Specify the dashboard and widget name in the prompt question (will use the data model connected to the dashboard) ([summarize dashboard sample healthcare widget top diagnosis])
1. Specify the dashboard, widget and model name in the prompt question (will use the data model in prompt) ([dashboard sample healthcare widget top diagnosis model sample healthcare])
1. apply validation on prompt entities and map to Sisense elements ([model sample healthcare] will be mapped to [Sample Healthcare]), this option can be skipped and is determined by the notebook parameter [to_validate_entities_from_question] if true, validation will occur.

    
---

#### Jupyter Notebooks
If you are not familiar with Jupyter Notebooks, we suggest reading about it [here](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/What%20is%20the%20Jupyter%20Notebook.html).

Get to know the Basics of Jupyter Notebooks, including how to add a New Notebook manually to the Jupyter Server [here](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html).

---

This Tutorial shows the following basic principles:
1. Examples for API parameters
1. How to parse the prompt question and extract sisense data info.
1. How to parse GPT response
1. how to return the response in html format

---
## Notebook Flow

![./BloxAIFlows-SummarizeWidget.png](./BloxAIFlows-SummarizeWidget.png)
---

## Content
1. [Test Cell](#Test-Cell)
1. [Imports](#Imports)
1. [Additional Parameters](#Additional-Parameters)
1. [Initialize Global Variables](#Initialize-Global-Variables)
2. [Search Fields In Sisense](#Search-Fields-In-Sisense)
3. [Ask AI Model Summary Questions](#Ask-AI-Model-Summary-Questions)
4. [Return HTML Result](#Return-HTML-Result)

## How to run the notebook
There are two options to execute this notebook, the first, which is the default one is via Jupyter server in Sisense instance. We refer this option as the "Remote" option.

The second one is "local" execution via local IDE.

The default behavior for the code in this notebook is to run in remote.
To run it locally, some code changes are required. To apply the changes, you can uncomment the code under the title - "for local develop".

The changes are in the following cells:
1. [Test Cell](#Test-Cell)
    Insert the IP of Sisense instance to environment variable - "API_GATEWAY_EXTERNAL_SERVICE_HOST"
    Insert the Port of Sisense instance to environment variable - "API_GATEWAY_EXTERNAL_SERVICE_PORT"
    Insert authentication token for Sisense user to environment variable - "Cookie" (How to get Sisense Authentication token is detailed explained [here](https://gitlab.sisense.com/SisenseTeam/DataSciense/bloxai/-/blob/master/README.md))
2. [Imports](#Imports)
    Change the import statements to local repository structure
3. [Additional Parameters](#Additional-Parameters)
    Change to extract authentication token from environment variable - "Cookie"

# Test Cell

The First notebook cell is a Test Cell. 

### What is a Test Cell ?
The additional parameters in the Test Cell are sample values. Its purpose is to mimic the additional parameters passed to the notebook from the Sisense Data Model build process. By having this test cell, the notebook can be run independently of the build process for debugging or editing purposes.   
When the notebook is executed from the Sisense build process, the test cell is skipped and the additional parameters are passed to the notebook downstream as a JSON string.  
You can change the Test cell location by changing its value in the notebook manifest. The default value is always the first notebook cell (cell 0).   

> "cellsDisable": [0]

# Imports

In [ ]:
import pandas as pd
import json
import re

# for remote develop
from AIIntegration import AIIntegration
from AIUtils import AIUtils
from InferenceQuestionType import QType

# for local develop
# from custom_code_notebooks.utils.AIIntegration import AIIntegration
# from custom_code_notebooks.utils.AIUtils import AIUtils
# from custom_code_notebooks.utils.InferenceQuestionType import QType

# Additional Parameters

Load the additional parameters passed from the query process or those initialized in the Test Cell above.

In this example multiple parameters are passed to the notebook:
1. question: a prompt question from the user ("please summarize dashboard sample healthcare widget top diagnosis")
1. widgetId: the widget id of the widget requesting the custom code notebook excecution
1. model_name: the model name of the dashboard (and widget) requesting the custom code notebook excecution 
1. to_validate_entities_from_question: boolean, true if validation on prompt entities required
1. cookie: sisense user authentication token

In [ ]:
print (additional_parameters)
add_param = json.loads(additional_parameters,strict=False)

orig_question = add_param['question']
model_name = add_param['model_name']
inputWidgetId = add_param['widgetId']

to_validate_entities_from_question = True
if "to_validate_entities_from_question" in add_param:
    to_validate_entities_from_question = add_param['to_validate_entities_from_question']

# for remote develop
cookie = add_param['cookie']
# for local develop
# cookie = os.getenv("Cookie")

# Initiaize global variables

the next cell will initialize the following variables:
**abort** - boolean value, indicate if the process got an exception or unexpected behavior and should stop execution.

**df_result** - dataFrame that will contain the notebook's output, the default value will indicate that 'Something went wrong'.

**model** - data model object- corresponding to model_name parameter.

**oai** - a connection to ai integration library

**utils** - a connection to utils library, contain a logger array that used as time tracking for performance logging. also contain a logger array that used as query tracker for both AI model and Jaql queries.


In [ ]:
abort = False
utils = AIUtils(inputWidgetId, model_name, add_param, cookie)
utils.add_time('Start')

df_result = pd.DataFrame(data=['Something went wrong'], columns=['Error'])

model = utils.get_model(model_name)
datamodelId = model['oid']

datasetId = model['datasets'][0]['oid']


utils.add_time('before import')
oai = AIIntegration(utils)
utils.add_time('After import')


utils.write_text_to_log_updates('<H1>AI start working on widget summary</H1>')

# Search Fields In Sisense
Query data from Sisense widget
## Steps:
1. Identify question type from prompt
2. Identify entities in the prompt
3. Get relevant dashboard/widget candidates from Sisense
4. Identify the candidates that best fit to prompt entities
5. Query data from Sisense

In [ ]:
options = [QType.TypeDashboardWidget,QType.TypeModelDashboardWidget]
if not to_validate_entities_from_question:
    options = [QType.TypeDashboardWidget]
[jaql, info, question_type_object] = oai.get_elements(model_name,orig_question,options, to_validate_entities_from_question)
utils.write_log_updates()
doid = question_type_object.sisense_dashboard_element["doid"]
woid = question_type_object.sisense_widget_element["woid"]

if info[0].startswith('Error'):
    df_result = oai.handle_error_in_element_extraction(info[0])
    

# Ask AI Model Summary Questions


In [ ]:
is_indicator_widget_data = False
dashboard_name = utils.get_dashboard_title(doid)
wtype, widget_title = utils.get_widget_type_and_title(doid, woid)

# data from indicator widget require special parsing      
if wtype=='indicator':
    is_indicator_widget_data = True  
abort, df_result = oai.get_df_from_jaql(jaql, model_name, is_indicator_widget_data) 
df_result_as_csv = df_result.to_csv(index=False)

if not abort:
    response = oai.get_widget_summary(df_result_as_csv, wtype, widget_title, dashboard_name)    
    print (response)

# Return HTML Result


In [ ]:
vec = [re.sub(r'\n',r'<br>',response)]
df_result = pd.DataFrame(data=vec, columns=['Value'])

df_result = utils.generate_html_result(re.sub(r'\n',r'<br>',response), df_result)

In [ ]:
df_result

In [ ]:
# from IPython.display import display, HTML
# display(HTML("<html>" + df_result['output'][0]+"</html>"))